In [28]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [29]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,51226744105,17058139939,879928209,879661334,437113589,437113589,457663395,457663395,701451983,553419347
1,62842626119,17986755349,427024666,427024666,217247624,217247624,232419725,232419725,1047830240,1045461045
2,57634824438,17209956752,634799444,634799444,357482882,357482882,448397177,448397177,915236773,642126921
3,50119284098,18286733290,259421011,259421011,102006337,102006337,109405616,109405616,713417381,705298630
4,47545603314,17339215524,878082456,875081378,418170793,418170793,467163218,467163218,511714488,509569794


In [30]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [31]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity


In [32]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [33]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [34]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0247 - val_loss: 0.0336
Epoch 2/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0212 - val_loss: 0.0336
Epoch 3/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0208 - val_loss: 0.0333
Epoch 4/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0207 - val_loss: 0.0336
Epoch 5/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0205 - val_loss: 0.0331
Epoch 6/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0205 - val_loss: 0.0335
Epoch 7/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0204 - val_loss: 0.0335
Epoch 8/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0204 - val_loss: 0.0337
Epoch 9/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0203 - val_loss: 0.0337
Epoch 10/20
2000/2000 [=====

In [35]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[28117743919, 554516117, 283582751, 309281138, 510198997]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


[0.92311895 0.00899014 0.0020754  0.00304362 0.0138314 ]
[0.93631424 0.01199095 0.00533862 0.00493588 0.01651228]
[9.3294024e-01 6.2869899e-03 7.3225424e-04 1.4588470e-03 1.7039511e-02]
[0.86696078 0.04375407 0.02090854 0.02335816 0.02547849]


In [36]:
# Save trained model
model.save("model.h5")
